In [16]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np

In [22]:
forexData = requests.get("https://assets.ino.com/data/minute/?s=FOREX_EURUSD&b=2021-2-23&f=json")
forexJson = forexData.json()
price = pd.DataFrame(forexJson, columns=["time", "Open", "High", "Low", "Close"])
price

,time,Open,High,Low,Close
0,1614056400000,1.217200,1.217350,1.217150,1.217205
1,1614056460000,1.217250,1.217350,1.217170,1.217250
2,1614056520000,1.217215,1.217215,1.217215,1.217215
3,1614056580000,1.217185,1.217195,1.217185,1.217195
4,1614056640000,1.217125,1.217125,1.217125,1.217125
...,...,...,...,...,...
5482,1614468240000,1.207445,1.207445,1.207445,1.207445
5483,1614469740000,1.207445,1.207445,1.207445,1.207445
5484,1614471240000,1.207445,1.207445,1.207445,1.207445
5485,1614472740000,1.207445,1.207445,1.207445,1.207445


In [23]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
from statistics import mean

def mySMA(myArr, period):
    return pd.Series(myArr).rolling(period).mean()
            
def myEMA(myArr, period):
    return pd.Series(myArr).ewm(adjust=False, span=period).mean()
    
def myMacd(myArr):
    twelve = myEMA(myArr, 12) 
    twentySix = myEMA(myArr, 26)
    macd = twelve-twentySix
#     print(macd, twelve, twentySix)
    return macd

def myMacdSignal(myArr):
    return myEMA(myMacd(myArr), 9)

def myMacdHist(myArr):
    return myMacd(myArr) - myMacdSignal(myArr)
    
class SmaCross(Strategy):
    def init(self):
        tempprice = self.data.Close
        print("buy and hold range", tempprice[0], tempprice[-1])
        self.ma1 = self.I(myEMA, tempprice, 10)
        self.ma2 = self.I(myEMA, tempprice, 30)
#         self.ma3 = self.I(myEMA, tempprice, 100)

        self.macd = self.I(myMacd, tempprice)
        self.macdSignal = self.I(myMacdSignal, tempprice)
        self.macdHist = self.I(myMacdHist, tempprice)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(price, SmaCross,
              exclusive_orders=True)
stats = bt.run()
bt.plot()

<ipython-input-23-2d9245995cb2>:44: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(price, SmaCross,


buy and hold range 1.217205 1.207445
